In [197]:
#pip install pyspark

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext 
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.master("local[*]").appName('Test').getOrCreate()
#spark = SparkSession.builder.master("http://10.0.2.15:4040").getOrCreate()

22/11/13 22:14:13 WARN Utils: Your hostname, Spark resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/11/13 22:14:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/13 22:14:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/13 22:14:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/13 22:14:15 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/11/13 22:14:15 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/11/13 22:14:15 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/11/13 22:14:15 WARN 

In [2]:
from pyspark.sql.functions import col
#Columns = TransactionID, TransactionTotal, TransactionNumberItems, TransactionDescription
df = spark.read.csv("/home/vboxuser/Downloads/Transactions.csv")
df = df.select(col("_c0").alias("ID"),col("_c1").alias("custID"), col("_c2").alias("TransTotal"), col("_c3").alias("TransNumItems"), col("_c4").alias("TransDesc"))
#df.withColumnRenamed(_c1, TransID)
#df.withColumnRenamed(_c2, TransTotal)
#df.withColumnRenamed(_c3, TransNumItems)
#df.withColumnRenamed(_c4, TransDesc)
df.show(n=5)
#df.printSchema()

+---+------+----------+-------------+--------------------+
| ID|custID|TransTotal|TransNumItems|           TransDesc|
+---+------+----------+-------------+--------------------+
|  1| 18329| 860.84564|            7|7exl8be8ncx3j92m8...|
|  2| 24446| 884.15656|            8|b3eu94yvb30v2fro2...|
|  3| 48269| 236.79364|            5|a7sb05tss5oe7zepd...|
|  4|  1848|  363.5627|            1|eeryin70d2or9o70g...|
|  5| 14543| 859.55756|            7|d6pblzstl0ea05jtt...|
+---+------+----------+-------------+--------------------+
only showing top 5 rows



In [3]:
#Problem 1 filter out transactions whose total amount is less than $200
T1 = df.filter(df.TransTotal >= 200)
T1.head(5)
#configure more driver memory to use .collect()

[Row(ID='1', custID='18329', TransTotal='860.84564', TransNumItems='7', TransDesc='7exl8be8ncx3j92m8xyzvlm0g2ztodzf40q4'),
 Row(ID='2', custID='24446', TransTotal='884.15656', TransNumItems='8', TransDesc='b3eu94yvb30v2fro2xk4184545ptgdj351jle2jtjx1a'),
 Row(ID='3', custID='48269', TransTotal='236.79364', TransNumItems='5', TransDesc='a7sb05tss5oe7zepda2ztu34r'),
 Row(ID='4', custID='1848', TransTotal='363.5627', TransNumItems='1', TransDesc='eeryin70d2or9o70g4k33o65'),
 Row(ID='5', custID='14543', TransTotal='859.55756', TransNumItems='7', TransDesc='d6pblzstl0ea05jttj2hcl6')]

In [4]:
from pyspark.sql import functions
#Problem 2: Over T1, group transactions by Num of Items, for each group
#calculate the sum of total amounts, avg total amounts, and min/max of total amounts
T2 = T1.groupby(T1.TransNumItems).agg(functions.sum('TransTotal'), functions.avg('TransTotal'), functions.min('TransTotal'), functions.max('TransTotal'))
T2.show(3)

+-------------+--------------------+-----------------+---------------+---------------+
|TransNumItems|     sum(TransTotal)|  avg(TransTotal)|min(TransTotal)|max(TransTotal)|
+-------------+--------------------+-----------------+---------------+---------------+
|            1|2.4303443106379622E8|600.7658829730391|     1000.00055|      999.99805|
|           10|2.4296692372027037E8|600.2770142165698|      1000.0008|      999.99854|
|            2|2.4303850916787988E8| 600.756658364223|      1000.0022|      999.99994|
+-------------+--------------------+-----------------+---------------+---------------+
only showing top 3 rows



In [5]:
#Problem 3: Report back T2 to the client side. Can I use any action?
T2.collect()
#T2.saveAsTextFile('T2.txt')

[Row(TransNumItems='1', sum(TransTotal)=243034431.06379622, avg(TransTotal)=600.7658829730391, min(TransTotal)='1000.00055', max(TransTotal)='999.99805'),
 Row(TransNumItems='10', sum(TransTotal)=242966923.72027037, avg(TransTotal)=600.2770142165698, min(TransTotal)='1000.0008', max(TransTotal)='999.99854'),
 Row(TransNumItems='2', sum(TransTotal)=243038509.16787988, avg(TransTotal)=600.756658364223, min(TransTotal)='1000.0022', max(TransTotal)='999.99994'),
 Row(TransNumItems='3', sum(TransTotal)=242405695.92514977, avg(TransTotal)=600.7963237411823, min(TransTotal)='1000.00024', max(TransTotal)='999.99695'),
 Row(TransNumItems='4', sum(TransTotal)=242598496.94242895, avg(TransTotal)=599.872154094484, min(TransTotal)='1000.0006', max(TransTotal)='999.9952'),
 Row(TransNumItems='5', sum(TransTotal)=242662547.40079027, avg(TransTotal)=600.5458148360192, min(TransTotal)='1000.0035', max(TransTotal)='999.9991'),
 Row(TransNumItems='6', sum(TransTotal)=242778366.49551067, avg(TransTotal)=6

In [6]:
#Over df, group the transactions by custID, and for each group report custID and the transactions count
T3 = T1.groupby('custID').agg(functions.count('custID').alias('custIDCount1'), functions.count('TransNumItems').alias('CountOfTransactions'))
T3 = T3.select(col('custID').alias('custID1'), 'custIDCount1', 'CountOfTransactions')
T3.show(5)

+-------+------------+-------------------+
|custID1|custIDCount1|CountOfTransactions|
+-------+------------+-------------------+
|   2069|          85|                 85|
|  10351|          73|                 73|
|  33665|          85|                 85|
|   2136|          76|                 76|
|  23318|          87|                 87|
+-------+------------+-------------------+
only showing top 5 rows



In [7]:
#T4: Filter from df total amount less than $600
T4 = df.filter(df.TransTotal >= 600)#.show(5)
T4.show(5)

+---+------+----------+-------------+--------------------+
| ID|custID|TransTotal|TransNumItems|           TransDesc|
+---+------+----------+-------------+--------------------+
|  1| 18329| 860.84564|            7|7exl8be8ncx3j92m8...|
|  2| 24446| 884.15656|            8|b3eu94yvb30v2fro2...|
|  5| 14543| 859.55756|            7|d6pblzstl0ea05jtt...|
| 11| 30590| 934.67194|           10|va8j0j3mq2tq4cqdt...|
| 12| 22930| 815.92725|            9|mo8lii0vdtltp157i...|
+---+------+----------+-------------+--------------------+
only showing top 5 rows



In [8]:
#T5 over T4, group the transactions by custID and for each group report the custID and the transactions count
T5 = T4.groupby('custID').agg(functions.count('custID').alias('custIDCount2'), functions.count('TransNumItems').alias('CountOfTransactions'))
T5 = T5.select(col('custID').alias('custID2'), 'custIDCount2', 'CountOfTransactions')
T5.show(5)

+-------+------------+-------------------+
|custID2|custIDCount2|CountOfTransactions|
+-------+------------+-------------------+
|  33665|          52|                 52|
|   2136|          40|                 40|
|  23318|          38|                 38|
|  29549|          45|                 45|
|  11332|          42|                 42|
+-------+------------+-------------------+
only showing top 5 rows



In [9]:
#T6 select the customer IDs whose T5.count * 5 < T3.count
#Multiply the agg of customer ID count by 5, select customer ID and customer ID count
T5 = T5.select('custID2', 'custIDCount2').withColumn('custIDCount2', col('custIDCount2') * 5)#.select('custID2', 'custIDCount2')
#T5.show(2)
#Select the customerID and customer ID count from the T3 df
T3 = T3.select('custID1','custIDCount1')
#T5.join(T5, ['custID2'])
#Perform an inner join on customer ID for df T3 and T5
joined = T5.join(T3, T3['custID1'] == T5['custID2'])
#Drop the duplicate customer ID column
joined = joined.drop('custID1')
#Select the customer ID after checking if T5.count is less than T3.count
T6 = joined.select('custID2').where(joined.custIDCount2 < joined.custIDCount1)
#This comes up as blank! --> ASK ABOUT THIS
T6.show(2)

+-------+
|custID2|
+-------+
+-------+



In [10]:
#Report back T6 to the client side
T6.collect()
#T6.saveAsTextFile('T6.txt')

[]